In [20]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [21]:
train = pd.read_csv('playground-series-s5e6/new_train.csv')
test = pd.read_csv('playground-series-s5e6/new_test.csv')

In [22]:
train

,Temparature,Humidity,Moisture,Soil Type,Crop Type,Nitrogen,Potassium,Phosphorous,Fertilizer Name,crop_N_req,...,N_gap,P_gap,K_gap,N_abnormality,P_abnormality,K_abnormality,Crop_Soil,NPK_gap_product,Moisture_Temp,stress
0,37,70,36,Clayey,Sugarcane,36,4,5,28-28,50.0,...,-32.6,11.75,-247.30,-46.6,-15.75,-280.8,Sugarcane_Clayey,9.472826e+04,1332,19
1,27,69,65,Sandy,Millets,30,6,18,28-28,20.0,...,-600.0,-4.20,-98.00,-590.0,3.80,-92.0,Millets_Sandy,-2.469600e+05,1755,29
2,29,63,32,Sandy,Millets,24,12,16,17-17-17,20.0,...,-600.0,-4.20,-98.00,-596.0,1.80,-86.0,Millets_Sandy,-2.469600e+05,928,16
3,35,62,54,Sandy,Barley,39,12,4,10-26-26,110.0,...,-510.0,-14.20,-98.00,-581.0,-10.20,-86.0,Barley_Sandy,-7.097160e+05,1890,11
4,35,58,43,Red,Paddy,37,2,16,DAP,0.0,...,-72.0,-28.00,-123.50,-35.0,-12.00,-121.5,Paddy_Red,-2.489760e+05,1505,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
849995,32,71,61,Black,Tobacco,23,1,25,20-20,0.0,...,-88.5,-27.00,-199.00,-65.5,-2.00,-198.0,Tobacco_Black,-4.755105e+05,1952,27
849996,35,72,47,Loamy,Millets,38,1,32,17-17-17,20.0,...,-1480.0,1.40,-104.00,-1462.0,23.40,-103.0,Millets_Loamy,2.154880e+05,1645,14
849997,28,50,61,Sandy,Maize,10,11,14,14-35-14,0.0,...,-620.0,13.30,12.00,-610.0,-0.20,-87.0,Maize_Sandy,-9.895200e+04,1708,26
849998,29,57,63,Loamy,Ground Nuts,7,10,4,DAP,12.5,...,-1487.5,8.90,-92.75,-1493.0,-4.60,-94.0,Ground Nuts_Loamy,1.227894e+06,1827,21


In [23]:
test

,Temparature,Humidity,Moisture,Soil Type,Crop Type,Nitrogen,Potassium,Phosphorous,crop_N_req,crop_P_req,...,N_gap,P_gap,K_gap,N_abnormality,P_abnormality,K_abnormality,Crop_Soil,NPK_gap_product,Moisture_Temp,stress
0,31,70,52,Sandy,Wheat,34,11,24,140.0,0.0,...,-480.0,-14.20,-98.00,-586.0,9.80,-87.0,Wheat_Sandy,-667968.00000,1612,17
1,27,62,45,Red,Sugarcane,30,14,15,50.0,32.5,...,-22.0,4.50,-86.00,-42.0,-13.00,-109.5,Sugarcane_Red,8514.00000,1215,2
2,28,72,28,Clayey,Ground Nuts,14,15,4,12.5,17.5,...,-70.1,-3.25,-273.55,-68.6,-16.75,-269.8,Ground Nuts_Clayey,-62321.52875,784,29
3,37,53,57,Black,Ground Nuts,18,17,36,12.5,17.5,...,-76.0,-9.50,-187.75,-70.5,9.00,-182.0,Ground Nuts_Black,-135555.50000,2109,19
4,31,55,32,Red,Pulses,13,19,14,0.0,0.0,...,-72.0,-28.00,-123.50,-59.0,-14.00,-104.5,Pulses_Red,-248976.00000,992,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249995,26,66,30,Red,Sugarcane,14,7,18,50.0,32.5,...,-22.0,4.50,-86.00,-58.0,-10.00,-116.5,Sugarcane_Red,8514.00000,780,21
249996,33,62,55,Red,Pulses,28,14,7,0.0,0.0,...,-72.0,-28.00,-123.50,-44.0,-21.00,-109.5,Pulses_Red,-248976.00000,1815,12
249997,36,53,64,Black,Paddy,28,11,27,0.0,0.0,...,-88.5,-27.00,-199.00,-60.5,0.00,-188.0,Paddy_Black,-475510.50000,2304,26
249998,36,67,26,Clayey,Paddy,33,0,10,0.0,0.0,...,-82.6,-20.75,-284.80,-49.6,-10.75,-284.8,Paddy_Clayey,-488132.96000,936,26


In [24]:
train['Fertilizer Name'].dtype == np.object_

True

In [25]:
target = ['Fertilizer Name']
cat_columns = [col for col in train.columns 
               if (train[col].dtype == 'object' or str(train[col].dtype) == 'category') 
               and col not in target]
num_columns = [col for col in train.columns if col not in cat_columns + target]

In [26]:
for i in cat_columns:
    print(f'Unique values in cat column: {i} - {train[i].nunique()}')
for i in cat_columns[:-1]:
    print(f'Unique values in cat column: {i} - {test[i].nunique()}')

Unique values in cat column: Soil Type - 5
Unique values in cat column: Crop Type - 11
Unique values in cat column: Crop_Soil - 55
Unique values in cat column: Soil Type - 5
Unique values in cat column: Crop Type - 11


In [27]:
target_col = target[0]  # unpack string from list

# Encode target separately
target_le = LabelEncoder()
train[target_col] = target_le.fit_transform(train[target_col].astype(str))

# Encode categorical features
encoders = {}

for col in cat_columns:
    if col == target_col:
        continue  # skip the target column

    if col in train.columns:
        le = LabelEncoder()
        train[col] = le.fit_transform(train[col].astype(str))
        train[col] = train[col].astype('category')
        encoders[col] = le

        if col in test.columns:
            test[col] = test[col].astype(str)
            test[col] = test[col].apply(lambda x: le.transform([x])[0] if x in le.classes_ else -1)
            test[col] = test[col].astype('category')
        else:
            print(f"Warning: Column '{col}' not found in test")
    else:
        print(f"Warning: Column '{col}' not found in train")


In [28]:
def mapk(actual, predicted, k=3):
    def apk(a, p, k):
        p = p[:k]
        score = 0.0
        hits = 0
        seen = set()
        for i, pred in enumerate(p):
            if pred in a and pred not in seen:
                hits += 1
                score += hits / (i + 1.0)
                seen.add(pred)
        return score / min(len(a), k)
    return np.mean([apk(a, p, k) for a, p in zip(actual, predicted)])

In [29]:
X = train.drop(['Fertilizer Name'],axis = 1)
y = train["Fertilizer Name"]

In [30]:
FOLDS = 5
skf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=42)
##  1
oof_xgb_1 = np.zeros(shape = (len(train) ,y.nunique()))
pred_prob_xgb_1 = np.zeros(shape = (len(test),y.nunique()))
##  2 
oof_xgb_2 = np.zeros(shape = (len(train) ,y.nunique()))
pred_prob_xgb_2 = np.zeros(shape = (len(test),y.nunique()))
##  3
oof_xgb_3 = np.zeros(shape = (len(train) ,y.nunique()))
pred_prob_xgb_3 = np.zeros(shape = (len(test),y.nunique()))


xgb_model_1 = XGBClassifier(
    max_depth=12,
    colsample_bytree=0.467,
    subsample=0.86,
    n_estimators=4000,
    learning_rate=0.03,
    gamma=0.26,
    max_delta_step=4,
    reg_alpha= 2.7,
    reg_lambda= 1.4,
    early_stopping_rounds=100,
    objective='multi:softprob',
    random_state = 42,
    eval_metric='mlogloss',
    enable_categorical=True)

xgb_model_2 = XGBClassifier(
    max_depth=13,
    colsample_bytree=0.4309,
    subsample=0.7281,
    n_estimators=1500,
    learning_rate=0.0179,
    gamma=0.0895,
    max_delta_step=4,
    reg_alpha= 0.4187,
    reg_lambda= 2.4665,
    early_stopping_rounds=100,
    objective='multi:softprob',
    eval_metric='mlogloss',
    random_state = 13,
    enable_categorical=True)

xgb_model_3 = XGBClassifier(
    max_depth=9,
    colsample_bytree=0.4930,
    subsample=0.9753,
    n_estimators=2353,
    learning_rate=0.0158,
    gamma=0.0035,
    max_delta_step=1,
    reg_alpha= 0.5994,
    reg_lambda= 4.5972,
    early_stopping_rounds=100,
    eval_metric='mlogloss',
    objective='multi:softprob',
    random_state = 25,
    enable_categorical=True)

correlations = []

for i, (train_idx, valid_idx) in enumerate(skf.split(X,y)):
    print('#' * 15, i+1, '#' *15)
    x_train, x_valid = X.iloc[train_idx],X.iloc[valid_idx]
    y_train, y_valid = y.iloc[train_idx],y.iloc[valid_idx]
    
    ##XGB 1 
    xgb_model_1.fit(x_train,y_train, eval_set=[(x_valid,y_valid)],verbose = 0)
    oof_xgb_1[valid_idx] = xgb_model_1.predict_proba(x_valid)
    pred_prob_xgb_1 +=xgb_model_1.predict_proba(test)

    top_3_preds_xgb_1 = np.argsort(oof_xgb_1[valid_idx], axis=1)[:, -3:][:, ::-1]  
    actual = [[label] for label in y_valid]
    map3_score_xgb_1 = mapk(actual, top_3_preds_xgb_1)
    print(f"✅ FOLD {i+1}: MAP@3 XGB_1 Score: {map3_score_xgb_1:.5f}")

    ##XGB 2 
    xgb_model_2.fit(x_train,y_train, eval_set=[(x_valid,y_valid)],verbose = 0)
    oof_xgb_2[valid_idx] = xgb_model_2.predict_proba(x_valid)
    pred_prob_xgb_2 +=xgb_model_2.predict_proba(test)

    top_3_preds_xgb_2 = np.argsort(oof_xgb_2[valid_idx], axis=1)[:, -3:][:, ::-1]  
    actual = [[label] for label in y_valid]
    map3_score_xgb_2 = mapk(actual, top_3_preds_xgb_2)
    print(f"☑️ FOLD {i+1}: MAP@3 XGB_2 Score: {map3_score_xgb_2:.5f}")

    ##XGB 2 
    xgb_model_3.fit(x_train,y_train, eval_set=[(x_valid,y_valid)],verbose = 0)
    oof_xgb_3[valid_idx] = xgb_model_3.predict_proba(x_valid)
    pred_prob_xgb_3 +=xgb_model_3.predict_proba(test)

    top_3_preds_xgb_3 = np.argsort(oof_xgb_3[valid_idx], axis=1)[:, -3:][:, ::-1]  
    actual = [[label] for label in y_valid]
    map3_score_xgb_3 = mapk(actual, top_3_preds_xgb_3)
    print(f"✔️ FOLD {i+1}: MAP@3 XGB_3 Score: {map3_score_xgb_3:.5f}")
    
    corr_12 = np.corrcoef(oof_xgb_1[valid_idx].ravel(), oof_xgb_2[valid_idx].ravel())[0,1]
    corr_13 = np.corrcoef(oof_xgb_1[valid_idx].ravel(), oof_xgb_3[valid_idx].ravel())[0,1]
    corr_23 = np.corrcoef(oof_xgb_2[valid_idx].ravel(), oof_xgb_3[valid_idx].ravel())[0,1]
    
    print(f"🔗 FOLD {i+1}:")
    print(f'Corr XGB_1 vs XGB_2: {corr_12:.5f}')
    print(f'Corr XGB_1 vs XGB_3: {corr_13:.5f}')
    print(f'Corr XGB_2 vs XGB_3: {corr_23:.5f}')
    correlations.append((corr_12 + corr_13 + corr_23) / 3)


print(f"\n📊 Mean after 5 Folds: {np.mean(correlations):.5f}")


############### 1 ###############
✅ FOLD 1: MAP@3 XGB_1 Score: 0.34152
☑️ FOLD 1: MAP@3 XGB_2 Score: 0.34140
✔️ FOLD 1: MAP@3 XGB_3 Score: 0.34343
🔗 FOLD 1:
Corr XGB_1 vs XGB_2: 0.94209
Corr XGB_1 vs XGB_3: 0.94973
Corr XGB_2 vs XGB_3: 0.93622
############### 2 ###############
✅ FOLD 2: MAP@3 XGB_1 Score: 0.34046
☑️ FOLD 2: MAP@3 XGB_2 Score: 0.34029
✔️ FOLD 2: MAP@3 XGB_3 Score: 0.34335
🔗 FOLD 2:
Corr XGB_1 vs XGB_2: 0.94159
Corr XGB_1 vs XGB_3: 0.94992
Corr XGB_2 vs XGB_3: 0.93576
############### 3 ###############
✅ FOLD 3: MAP@3 XGB_1 Score: 0.34162
☑️ FOLD 3: MAP@3 XGB_2 Score: 0.34184
✔️ FOLD 3: MAP@3 XGB_3 Score: 0.34516
🔗 FOLD 3:
Corr XGB_1 vs XGB_2: 0.94116
Corr XGB_1 vs XGB_3: 0.95024
Corr XGB_2 vs XGB_3: 0.93551
############### 4 ###############
✅ FOLD 4: MAP@3 XGB_1 Score: 0.34194
☑️ FOLD 4: MAP@3 XGB_2 Score: 0.34157
✔️ FOLD 4: MAP@3 XGB_3 Score: 0.34443
🔗 FOLD 4:
Corr XGB_1 vs XGB_2: 0.94239
Corr XGB_1 vs XGB_3: 0.95019
Corr XGB_2 vs XGB_3: 0.93564
############### 5 ######

In [31]:
actual = [[label] for label in y]

# XGB_1
top_3_preds_xgb_1 = np.argsort(oof_xgb_1, axis=1)[:, -3:][:, ::-1]  
map3_score_xgb_1 = mapk(actual, top_3_preds_xgb_1)
print(f'✅ Final XGB_1 MAP@3 Score: {map3_score_xgb_1:.5f}')

# XGB_2
top_3_preds_xgb_2 = np.argsort(oof_xgb_2, axis=1)[:, -3:][:, ::-1]  
map3_score_xgb_2 = mapk(actual, top_3_preds_xgb_2)
print(f'✅ Final XGB_2 MAP@3 Score: {map3_score_xgb_2:.5f}')

# XGB_3
top_3_preds_xgb_3 = np.argsort(oof_xgb_3, axis=1)[:, -3:][:, ::-1]  
map3_score_xgb_3 = mapk(actual, top_3_preds_xgb_3)
print(f'✅ Final XGB_3 MAP@3 Score: {map3_score_xgb_3:.5f}')

✅ Final XGB_1 MAP@3 Score: 0.34137
✅ Final XGB_2 MAP@3 Score: 0.34112
✅ Final XGB_3 MAP@3 Score: 0.34400


In [32]:
pred_prob_xgb_1 /= FOLDS
pred_prob_xgb_2 /=FOLDS
pred_prob_xgb_3 /= FOLDS

In [33]:
w3= 0.4714285714285714
w1=0.2666666666666667
w2=0.26190476190476191

pred_prob = w1 * pred_prob_xgb_1 + w2 *pred_prob_xgb_2 + w3 * pred_prob_xgb_3

top_3_preds = np.argsort(pred_prob, axis=1)[:, -3:][:, ::-1]
top_3_labels = target_le.inverse_transform(top_3_preds.ravel()).reshape(top_3_preds.shape)
df_sub = pd.read_csv("playground-series-s5e6/sample_submission.csv")
submission = pd.DataFrame({
    'id': df_sub['id'],
    'Fertilizer Name': [' '.join(row) for row in top_3_labels]
})
submission.to_csv('Fet_3xgboost_submission.csv', index=False)
print("✅ Submission file saved as 'Fet_3xgboost_submission.csv'")

submission.head()

✅ Submission file saved as 'Fet_3xgboost_submission.csv'


,id,Fertilizer Name
0,750000,28-28 DAP 17-17-17
1,750001,17-17-17 Urea 20-20
2,750002,20-20 10-26-26 28-28
3,750003,14-35-14 Urea 17-17-17
4,750004,20-20 10-26-26 Urea


In [34]:
n_classes = 7

df = pd.DataFrame(pred_prob, columns=[f'class_{i}' for i in range(n_classes)])
df.to_csv('Fet_3xgboost_prob.csv', index=False)